In [ ]:
# TODO: deprecate...
# this is just a temp fix to get things working locally. use the server config when starting jupyter
import sys, os

# NOTE: change this path if you need to for your system
client_side_path = os.getcwd()
print("client_side_path: %s" % (client_side_path))
sys.path.append(client_side_path)

In [ ]:
from wsclient.client import TestWebSocketClient
from tornado import ioloop

print("IOLoop: %s" % (ioloop.IOLoop.current()))
client = TestWebSocketClient()
client.connect('ws://localhost:8675/ws')

In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import json
import random

class Plotter():
    """
    Wrapper class for the plt import with some helpers and a callback for the client.
    """
    bwidth = 0.5
    
    def __init__(self):
        """
        Initialize the object
        """
        # values sent mapped to the number of times sent
        self.val_counts = {}

        # width of the bars for the chart
        self.fig = None
        self.ax = None
        self.rects = []
        self.colors = {}
    
    def random_color(self):
        """
        Generate a random color (hex string).
        """
        lvls = range(32,256,32)
        t = tuple(random.choice(lvls) for _ in range(3))
        return "#%02x%02x%02x" % t

    def labelrects(self):
        """
        Helper for labelling the bars
        """
        # label the bars
        for rect in self.rects:
            height = rect.get_height()
            self.ax.text(rect.get_x() + rect.get_width()/2., 
                    0,
                    '%d' % int(height),
                    ha='center', 
                    va='bottom')


    def setup_plot(self):
        """
        Initialize the plot
        """
        self.fig, self.ax = plt.subplots()

        # add some text for y label and title
        self.ax.set_ylabel('Counts')
        self.ax.set_title('Counts of values sent to the WSServer')

        self.update_plot()

    
    def update_plot(self):
        """
        Update the plot with the current values of val_counts
        """
        plt.cla()
        vals = tuple(k for k,v in self.val_counts.items())
        counts = [v for k,v in self.val_counts.items()]
        
        for v in vals:
            self.colors.setdefault(v, self.random_color())
        
        # x locations for counts
        ind = np.arange(len(self.val_counts))

        self.rects = self.ax.bar(ind, counts, Plotter.bwidth, color=[c for v, c in self.colors.items()], align='center')

        # add some text for ticks
        self.ax.set_xticks(ind)
        self.ax.set_xticklabels(vals)

        # if we have some rects, set their heigthts, label
        if len(self.rects) > 0:
            for rect, h in zip(self.rects, counts):
                rect.set_height(h)
            self.ax.legend((self.rects), tuple('%s counts' % (v) for v in vals))
            self.labelrects()
        
        # redraw
        self.fig.canvas.draw()

    
    def client_msg_callback(self, msg):
        """
        Callback for the client to call when a message is received.
        """
        # TODO: Change msg to be a dict or other object before getting here...
        d = json.loads(msg.replace("'", "\""))
        if 'value' in d:
            # message contains a 'value' key. use it's value to update
            # the val_counts dict.
            self.val_counts[d['value']] = self.val_counts.setdefault(d['value'], 0) + 1
            self.update_plot()

p = Plotter()
# Better way to do this? callback feel wrong...
client.msg_callback(p.client_msg_callback)
p.setup_plot()

In [ ]:
from ipywidgets import Button, IntSlider
from IPython.display import display

button = Button(description='Send a test Message!')
valinput = IntSlider(description='Enter a number to send', min=0, max=10)

display(valinput)
display(button)

def send_test_msg(b):
    d = {
        'does_it': 'blend',
        'value': valinput.value,
    }
    print('send_test_msg: sending test %s' % (d))
    client.send(d)

button.on_click(send_test_msg)

In [ ]:
p.val_counts